# 필요한 모듈 및 데이터 오픈

In [1]:
import glob
import os
import re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw) #extend(raw) : list끝에 raw의 모든 항목을 넣는다.(append()와 차이가 있음)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 데이터 전처리

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
#    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

# 토큰화

In [5]:
def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>",
        
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen = 15,padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc7ac0ed100>


In [6]:
print(tensor[:3, :10])

[[   2   50    5   91  297   65   57    9  969 6042]
 [   2   17 2639  873    4    8   11 6043    6  329]
 [   2   36    7   37   15  164  282   28  299    4]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
src_input = tensor[:,:-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


# 데이터셋 생성

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size= 0.2)

In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

vocab_size= tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 모델 생성

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 모델 빌드
    
    - Sequentail 모델이나, Funtional API는 선언형(Define-and-Run)이므로 모델을 정의&인스턴스화한 단계에서 계산 그래프가 정적으로 구축된다. 하지만 Subclassing 모델은 명령형(Define-by-Run)이므로 실행한 단계에서 계산 그래프가 동적으로 구축된다.
      
    - 즉, 추론을 1 회 시행하여 계산 그래프를 동적으로 실행시켜주어야 한다.
    
    - 추론을 1회 시행히키지 않을 경우:
    "This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build."
    이런 오류가 뜬다.
      

In [13]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.90532655e-04, -2.95985719e-05,  1.02315775e-04, ...,
         -1.98390073e-04,  1.11468966e-04, -1.51840490e-04],
        [ 4.45410900e-04,  2.41103087e-04,  8.71573357e-05, ...,
         -3.80338111e-04,  1.03581748e-04, -3.42435349e-04],
        [ 2.70226330e-04,  5.49512217e-04, -1.35619048e-04, ...,
         -5.85307018e-04,  1.69601844e-04, -3.21467844e-04],
        ...,
        [ 7.03768979e-04,  2.24478907e-04, -1.34904403e-03, ...,
          3.27190210e-05,  2.18600369e-04, -5.60812186e-04],
        [ 3.16325459e-04, -3.56032615e-05, -1.36740785e-03, ...,
          4.22212906e-04, -9.67391770e-06, -6.41252787e-04],
        [ 2.44763534e-04, -2.56535539e-04, -1.16847444e-03, ...,
          6.93315582e-04,  2.84483067e-05, -4.97869274e-04]],

       [[-9.41432663e-05, -2.53691396e-04, -1.35166949e-04, ...,
         -2.35344196e-04, -3.88072694e-05,  1.88157705e-04],
        [ 1.60468917e-04, -4.93358821e-04,  9

In [14]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


# 모델 훈련

In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 123s 176ms/step - loss: 3.6010
Epoch 2/10
686/686 [==============================] - 125s 182ms/step - loss: 3.1181
Epoch 3/10
686/686 [==============================] - 125s 182ms/step - loss: 2.9171
Epoch 4/10
686/686 [==============================] - 125s 182ms/step - loss: 2.7621
Epoch 5/10
686/686 [==============================] - 125s 182ms/step - loss: 2.6274
Epoch 6/10
686/686 [==============================] - 125s 182ms/step - loss: 2.5061
Epoch 7/10
686/686 [==============================] - 125s 182ms/step - loss: 2.3947
Epoch 8/10
686/686 [==============================] - 125s 182ms/step - loss: 2.2914
Epoch 9/10
686/686 [==============================] - 125s 182ms/step - loss: 2.1939
Epoch 10/10
686/686 [==============================] - 125s 182ms/step - loss: 2.1016


# 결과 출력

In [21]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [23]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much <end> '

# 회고

처음 인공지능을 배울때는 모델이라는 말을 많이 들어서 모델에 대해서만 열심히 봤었다.

그래서 코드들을 볼 때도 모델 부분을 가장 흥미롭게 봤었다.

지금도 매일매일 새로운 모델이 나오는 것으로 알고 있고,

내가 아는 모델보단 들어보지도 못한 모델이 훨씬 많은 것도 알고 있지만

이제는 적어도 코드로 모델을 봐도 어느정도 아 이런걸 썻구나 하는 생각은 든다.

이제 모델에 대한 코드가 조금 보여서 같은 모델로 성능을 올리기 위해 어떤 방법이 있을지 생각해보다가

데이터 전처리가 정말 중요하다는 생각에 이번 Exploration에서는 데이터 전처리에 대한 부분을 유심히 봤다.

오.. 하나도 이해 못했다,,,ㅠㅅㅠ

노드에 다양한 데이터에 대한 전처리기법들이 많으니 다시 보면서 복습해야겠다.